In [1]:
import tensorflow
import pandas as pd
import numpy as np
import pickle
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

/home/codekhal/miniconda3/envs/av/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/codekhal/miniconda3/envs/av/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/codekhal/miniconda3/envs/av/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/codekhal/miniconda3/envs/av/lib/python3.5/s

In [2]:
model=tensorflow.keras.models.load_model(r"/home/codekhal/virtual-tourist-guide/SIHchatbot2020.h5")
with open(r"/home/codekhal/virtual-tourist-guide/vocab.p","rb") as f:
    words,classes=pickle.load(f)
df=pd.read_json("intents.json")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    print()
    return(np.array(bag))

In [4]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [10]:
import numpy as np
result=classify_local("tourism in goa")
#print(result)
for intent in df['intents']:
    if float(result[0][1])>0.6:
        if intent['tag']==result[0][0]:
            print(np.random.choice(intent['responses']))
    else:
        if intent['tag']=='noanswer':
            print(np.random.choice(intent['responses']))

found in bag: tour
found in bag: in
found in bag: goa

Palolem, Siolim, Patto Centre, Carambolim, Gumal Vaddo, Calangute, Dmello Vaddo, South Goa, Sangolda, Fatona, Ilhas, Defence Colony, Fatorda, Balcbot, Calata
